# gebrauchte Bücher aus Goodreads to-read liste suchen

In [1]:
import numpy as np
import csv
import os
import datetime as dt
from matplotlib import pyplot as plt
import pandas as pd
from bs4 import BeautifulSoup
import requests
from tqdm.notebook import tqdm
tqdm.pandas()

In [2]:
TOTAL_NUMBER = 300  # max number of books to search on Booklooker
START_DATE = pd.Timestamp('2010-01-01') # Books added to to-read shelf after this date will be considered
DATE = str(dt.datetime.today().date()) # in- and output datestamp
print(DATE)

2021-03-18


# WIP
## goodreads export csv

In [16]:
# #Simple assignment
# from selenium.webdriver import Edge

# driver = Edge()
# #Or use the context manager
# from selenium.webdriver import Edge

# with Edge() as driver:
#    #your code inside this indent
#    pass


## goodreads CSV-Datei parsen

In [61]:
csv_path = os.path.join(os.getcwd(), f'goodreads_library_export_{DATE}.csv')

df = pd.read_csv(csv_path, index_col='Book Id',
dtype={"Title": str, "Author": str, "Author l-f": str, "Additional Authors": str, "ISBN":str,"ISBN-13":str, "Publisher": str, "Binding": str, 'Bookshelves': 'category', 'Exclusive Shelf': 'category', },
parse_dates=["Original Publication Year", 'Date Read', 'Date Added', "Year Published"])
df.head(10)

,Title,Author,Author l-f,Additional Authors,ISBN,ISBN13,My Rating,Average Rating,Publisher,Binding,...,Private Notes,Read Count,Recommended For,Recommended By,Owned Copies,Original Purchase Date,Original Purchase Location,Condition,Condition Description,BCID
Book Id,,,,,,,,,,,,,,,,,,,,,
18721991,"Doctor Sleep (The Shining, #2)",Stephen King,"King, Stephen",NaN,"=""3453268555""","=""9783453268555""",0,4.11,Heyne,Hardcover,...,ca. S. 370,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
28186015,Weapons of Math Destruction: How Big Data Incr...,Cathy O'Neil,"O'Neil, Cathy",NaN,"=""0553418815""","=""9780553418811""",0,3.88,Crown,Hardcover,...,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
55848259,Komplett Gänsehaut,Sophie Passmann,"Passmann, Sophie",NaN,"=""""","=""9783462053616""",0,3.73,Kiepenheuer&Witsch,Hardcover,...,NaN,1,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
52275335,How to Avoid a Climate Disaster: The Solutions...,Bill Gates,"Gates, Bill",NaN,"=""0241448301""","=""9780241448304""",3,4.21,Allen Lane,Hardcover,...,NaN,1,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
29780253,Born a Crime: Stories From a South African Chi...,Trevor Noah,"Noah, Trevor",NaN,"=""0385689225""","=""9780385689229""",5,4.46,Doubleday Canada,Hardcover,...,NaN,1,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
42242793,Kurze Antworten auf große Fragen,Stephen Hawking,"Hawking, Stephen",Hainer Kober,"=""""","=""""",4,4.30,Klett-Cotta,Kindle Edition,...,NaN,1,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
44492285,"Dune Messiah (Dune Chronicles, #2)",Frank Herbert,"Herbert, Frank",NaN,"=""0593098234""","=""9780593098233""",3,3.89,Ace Books,Paperback,...,NaN,1,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
44512537,Growth: From Microorganisms to Megacities,Vaclav Smil,"Smil, Vaclav",NaN,"=""0262042835""","=""9780262042833""",0,3.90,The MIT Press,Hardcover,...,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
4070074,Sustainable Energy - Without the Hot Air,David J.C. MacKay,"MacKay, David J.C.",NaN,"=""0954452933""","=""9780954452933""",0,4.40,UIT Cambridge Ltd.,Paperback,...,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN


In [5]:
df.shape

(376, 30)

In [6]:
df['ISBN'] = df['ISBN'].str.split('\"').str.get(1)
df['ISBN13'] = df['ISBN13'].str.split('\"').str.get(1)
df.head()

,Title,Author,Author l-f,Additional Authors,ISBN,ISBN13,My Rating,Average Rating,Publisher,Binding,...,Private Notes,Read Count,Recommended For,Recommended By,Owned Copies,Original Purchase Date,Original Purchase Location,Condition,Condition Description,BCID
Book Id,,,,,,,,,,,,,,,,,,,,,
18721991,"Doctor Sleep (The Shining, #2)",Stephen King,"King, Stephen",NaN,3453268555,9783453268555,0,4.11,Heyne,Hardcover,...,ca. S. 370,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
28186015,Weapons of Math Destruction: How Big Data Incr...,Cathy O'Neil,"O'Neil, Cathy",NaN,0553418815,9780553418811,0,3.88,Crown,Hardcover,...,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
55848259,Komplett Gänsehaut,Sophie Passmann,"Passmann, Sophie",NaN,,9783462053616,0,3.73,Kiepenheuer&Witsch,Hardcover,...,NaN,1,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
52275335,How to Avoid a Climate Disaster: The Solutions...,Bill Gates,"Gates, Bill",NaN,0241448301,9780241448304,3,4.21,Allen Lane,Hardcover,...,NaN,1,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
29780253,Born a Crime: Stories From a South African Chi...,Trevor Noah,"Noah, Trevor",NaN,0385689225,9780385689229,5,4.46,Doubleday Canada,Hardcover,...,NaN,1,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN


In [7]:
to_read = df[df['Exclusive Shelf'] == 'to-read']


In [8]:
to_read.sort_values('Date Added', ascending=False, inplace=True)
to_read_2021 = to_read[to_read['Date Added'] > START_DATE]

<ipython-input-8-d2f2cc0e2833>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to_read.sort_values('Date Added', ascending=False, inplace=True)


In [9]:
to_read_2021.head()

,Title,Author,Author l-f,Additional Authors,ISBN,ISBN13,My Rating,Average Rating,Publisher,Binding,...,Private Notes,Read Count,Recommended For,Recommended By,Owned Copies,Original Purchase Date,Original Purchase Location,Condition,Condition Description,BCID
Book Id,,,,,,,,,,,,,,,,,,,,,
28186015,Weapons of Math Destruction: How Big Data Incr...,Cathy O'Neil,"O'Neil, Cathy",NaN,0553418815,9780553418811,0,3.88,Crown,Hardcover,...,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
44512537,Growth: From Microorganisms to Megacities,Vaclav Smil,"Smil, Vaclav",NaN,0262042835,9780262042833,0,3.90,The MIT Press,Hardcover,...,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
4070074,Sustainable Energy - Without the Hot Air,David J.C. MacKay,"MacKay, David J.C.",NaN,0954452933,9780954452933,0,4.40,UIT Cambridge Ltd.,Paperback,...,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
56859211,Zahlen sind Waffen. Gespräche über die Zukunf...,Sibylle Berg,"Berg, Sibylle",Dietmar Dath,9783751805,,0,3.29,Matthes & Seitz,Paperback,...,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
16131209,The Stench of Honolulu: A Tropical Adventure,Jack Handey,"Handey, Jack",NaN,1455522384,9781455522385,0,3.34,Grand Central Publishing,Hardcover,...,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN


In [10]:
type(to_read_2021)

pandas.core.frame.DataFrame

## Beispiel
### Jetzt Suchmaschine mittels BeatifulSoup scrapen um Bestpreise für Bücher zu finden

In [17]:
#Testbuch
book = to_read_2021.iloc[6]
book

Title                                       Dreck
Author                                Bill Buford
Author l-f                           Buford, Bill
Additional Authors                  Sabine Hübner
ISBN                                   3446267719
ISBN13                              9783446267718
My Rating                                       0
Average Rating                               3.38
Publisher                                  Hanser
Binding                                 Hardcover
Number of Pages                             508.0
Year Published                2020-01-01 00:00:00
Original Publication Year                    2020
Date Read                                     NaT
Date Added                    2021-02-09 00:00:00
Bookshelves                               to-read
Bookshelves with positions         to-read (#244)
Exclusive Shelf                           to-read
My Review                                     NaN
Spoiler                                       NaN


In [18]:
url = 'https://www.booklooker.de/B%C3%BCcher/Angebote/isbn=9783446267718?sortOrder=preis_total'
search_url = 'https://www.booklooker.de/B%C3%BCcher/Angebote/'
base_url = 'https://www.booklooker.de'
query_parameters = {
    'isbn': book['ISBN13'],
    'sortOrder': 'preis_total'
}

## Kann geskippt werden

In [19]:
page = requests.get(url)
soup = BeautifulSoup(page.content)

table class="resultlist_products"

tbody

tr id="article_*ID*" class="articleRow resultlist_products*product*" ODER

articleRow resultlist_products*premium*

In [20]:
results = soup.find(class_='resultlist_products')
products = results.find_all('tr', class_='articleRow')


In [101]:
entries = []
for idx, product in enumerate(products):
    product_info = product.find("td", class_="resultlist_productsimage")
    price_info = product.find("td", class_="resultlist_productsright")
    title_info =product_info.find('h2', class_='unformated')
    book_title = title_info.find('span', class_='artikeltitel')
    link = base_url + title_info.a.attrs['href']

    author = product_info.find('span', class_='notranslate')
    condition = str(product_info.find(string=lambda text: 'zustand' in text.lower()))
    if condition:
        prefix = 'Zustand: '
        condition = condition[len(prefix):] if condition.startswith(prefix) else condition
    else:
        condition = str(product_info.find(string=lambda text: 'neuware' in text.lower()))
    price = price_info.find('span', class_='price')
    row = [book_title.text.strip(), author.text.strip(), price.text.strip()[:-2], condition.strip(), link]
    entries.append(row)
    # print(f"[{idx}]: title: {book_title.text.strip()}, author: {author.text.strip()}, price = {price.text.strip()}, state: {state.strip()}), link: {link}")


In [22]:
web = pd.DataFrame(entries, columns=['Title', 'Author', 'Price', 'Condition', 'URL'])
web.head()

,Title,Author,Price,Condition,URL
0,"Dreck - Wie ich meine Familie einpackte, Koch ...","Dreck - Wie ich meine Familie einpackte, Koch ...","16,00",leichte Gebrauchsspuren,https://www.booklooker.de/B%C3%BCcher/Bill-Buf...
1,"Dreck Wie ich meine Familie einpackte, Koch in...","Dreck Wie ich meine Familie einpackte, Koch in...","18,39",wie neu,https://www.booklooker.de/B%C3%BCcher/Bill-Ver...
2,"Dreck Wie ich meine Familie einpackte, Koch in...","Dreck Wie ich meine Familie einpackte, Koch in...","18,39",leichte Gebrauchsspuren,https://www.booklooker.de/B%C3%BCcher/Bill-Ver...
3,"Dreck Wie ich meine Familie einpackte, Koch in...","Dreck Wie ich meine Familie einpackte, Koch in...","18,39",leichte Gebrauchsspuren,https://www.booklooker.de/B%C3%BCcher/Bill-Ver...
4,"Dreck - Wie ich meine Familie einpackte, Koch ...","Dreck - Wie ich meine Familie einpackte, Koch ...","19,40",wie neu,https://www.booklooker.de/B%C3%BCcher/Bill-Buf...


## Funktionen
Build Search URL

In [23]:
from urllib.parse import urlencode, urljoin, quote_plus
def build_booklooker_url(**kwargs):
    isbn13 = kwargs.get('ISBN13', "")
    isbn = kwargs.get('ISBN', "")
    title = kwargs.get('Title', "")
    author = kwargs.get('Author', "")
    
    search_url = 'https://www.booklooker.de/B%C3%BCcher/Angebote/'
    payload = {'sortOrder': "preis_total"}
    if len(isbn13) > 0:
        payload['isbn'] = isbn13
    elif len(isbn) > 0:
        payload['isbn'] = isbn
    else:
        payload['autor'] = author
        payload['titel'] = title
        print(f'ISBN not found, using: {author}: {title}')

    url = build_url(search_url, payload)
    return url

def build_abebooks_url(**kwargs):
    #TODO
    raise NotImplementedError('not yet finished.')
    search_url = 'https://www.abebooks.de/servlet/SearchResults?sts=t&cm_sp=SearchF-_-home-_-Results&an=Author&tn=Title&kn=&isbn='
    isbn13 = kwargs.get('ISBN13', "")
    isbn = kwargs.get('ISBN', "")
    title = kwargs.get('Title', "")
    author = kwargs.get('Author', "")
    
    payload = {'sortOrder': "preis_total"}
    if len(isbn13) > 0:
        payload['isbn'] = isbn13
    elif len(isbn) > 0:
        payload['isbn'] = isbn
    else:
        payload['an'] = author
        payload['tn'] = title
        print(f'ISBN not found, using: {author}: {title}')

    url = build_url(search_url, payload)
    return url

def build_url(base_url, payload):
    result = urlencode(payload, quote_via=quote_plus)
    url = urljoin(base_url, result)
    return url

def get_soup(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content.decode('utf-8', 'ignore'))
    return soup

In [24]:
import re
non_decimal = re.compile(r'[^\d.]+')

Soup-Suchergebnisse auslesen

In [25]:
def get_booklooker_results(soup, limit=10, as_df=False, rating=None):
    entries = []
    results = soup.find(class_='resultlist_products')
    if results is None:
        return entries
    products = results.find_all('tr', class_='articleRow')
    for idx, product in enumerate(products):
        try:
            product_info = product.find("td", class_="resultlist_productsimage")
            title_info =product_info.find('h2', class_='unformated')
            book_title = title_info.find('span', class_='artikeltitel notranslate')
            link = base_url + title_info.a.attrs['href']

            author_info =product_info.find('h3', class_='unformated')
            author = author_info.find('span', class_='notranslate')

            condition = str(product_info.find(string=lambda text: 'zustand' in text.lower()))
            prefix = 'Zustand: '
            condition = condition[len(prefix):] if condition.startswith(prefix) else condition
            
            price_info = product.find("div", class_="productPrices")
            prices = price_info.find_all(string=lambda text: '€' in text.lower())
        except AttributeError as notfound:
            continue
        price_list = [s[:-1].strip() for s in prices]
        price = price_list[0] if len(price_list) > 0 else ""
        vsk = price_list[1] if len(price_list) > 1 else ""
        price = price.replace(",", ".")
        vsk = vsk.replace(",", ".")
        price = non_decimal.sub('', price)
        vsk = non_decimal.sub('', vsk)
        price = float(price) if len(price) > 0 else 0
        vsk = float(vsk) if len(vsk) > 0 else 0
        row = (book_title.text.strip(), author.text.strip(), price, vsk, condition.strip(), link)
        entries.append(row)
        if idx > (limit - 1):
            break
    if as_df:
        df = pd.DataFrame(entries, columns=['Title', 'Author', 'Price', 'VSK', 'Condition', 'URL'])
        if rating:
            df['Average Rating'] = rating
        return df
    else:
        return entries


## Bringing it all together
Für jedes Buch:
- URL erzeugen
- HTML-Suppe abrufen
- Suchergebnisse parsen
- DataFrame erzeugen

In [26]:
def get_sales_info_df(df, limit=10, as_df=True):
    url = build_booklooker_url(**df)
    soup = get_soup(url)
    search_results = get_booklooker_results(soup, limit=limit, as_df=as_df, rating=df['Average Rating'])
    if len(search_results) < 1:
        print(f"no search results found for {df['Title']}, ISBN: {df['ISBN']}")
    if as_df:
        return search_results
    else:
        return np.array(search_results)

In [27]:
def get_sales_info(title, isbn, isbn13, limit=10, as_df=True):
    isbn_ = isbn13 if len(isbn13) > 0 else isbn
    if len(isbn_) <= 0:
        print(f'{title}: no ISBN found!')
        return []
        # raise KeyError('No ISBN Found')
    d = {'ISBN': isbn_}
    url = build_booklooker_url(**d)
    soup = get_soup(url)
    search_results = get_booklooker_results(soup, limit=limit, as_df=as_df)
    return np.array(search_results)

In [28]:
# sales_info = []
my_df = to_read_2021.iloc[:TOTAL_NUMBER]
# Create and register a new `tqdm` instance with `pandas`
# (can use tqdm_gui, optional kwargs, etc.)
tqdm.pandas()

# Now you can use `progress_apply` instead of `apply`
sales_infos = my_df.progress_apply(get_sales_info_df, axis=1)


  0%|          | 0/253 [00:00<?, ?it/s]

no search results found for Weapons of Math Destruction: How Big Data Increases Inequality and Threatens Democracy, ISBN: 0553418815
no search results found for Growth: From Microorganisms to Megacities, ISBN: 0262042835
no search results found for Sustainable Energy - Without the Hot Air, ISBN: 0954452933
no search results found for Zahlen sind Waffen.  Gespräche über die Zukunft. (Fröhliche Wissenschaft Band 176), ISBN: 9783751805
no search results found for The Stench of Honolulu: A Tropical Adventure, ISBN: 1455522384
no search results found for Entangled Life: How Fungi Make Our Worlds, Change Our Minds & Shape Our Futures, ISBN: 0525510311
no search results found for The Shallows: What the Internet Is Doing to Our Brains, ISBN: 0393339750
ISBN not found, using: Oliver Stone: Chasing the Light: Writing, Directing, and Surviving Platoon, Midnight Express, Scarface, Salvador, and the Movie Game
no search results found for Chasing the Light: Writing, Directing, and Surviving Platoon,

In [29]:
sales_infos.iloc[7]

,Title,Author,Price,VSK,Condition,URL,Average Rating
0,DAVE | Roman | Raphaela Edelbauer | Buch | Deu...,Raphaela Edelbauer,25.0,0,None,https://www.booklooker.de/B%C3%BCcher/Raphaela...,3.72
1,DAVE,Raphaela Edelbauer,25.0,0,None,https://www.booklooker.de/B%C3%BCcher/Raphaela...,3.72
2,DAVE | Roman | Raphaela Edelbauer | Buch | Deu...,Raphaela Edelbauer,25.0,0,None,https://www.booklooker.de/B%C3%BCcher/Raphaela...,3.72
3,DAVE,Raphaela Edelbauer,25.0,0,None,https://www.booklooker.de/B%C3%BCcher/Raphaela...,3.72
4,DAVE,Raphaela Edelbauer,25.0,0,None,https://www.booklooker.de/B%C3%BCcher/Raphaela...,3.72
5,DAVE - Roman,Raphaela Edelbauer,25.0,0,None,https://www.booklooker.de/B%C3%BCcher/Raphaela...,3.72
6,DAVE,Raphaela Edelbauer,25.0,0,None,https://www.booklooker.de/B%C3%BCcher/Raphaela...,3.72
7,DAVE,Raphaela Edelbauer,25.0,0,None,https://www.booklooker.de/B%C3%BCcher/Raphaela...,3.72
8,DAVE,Raphaela Edelbauer,25.0,0,None,https://www.booklooker.de/B%C3%BCcher/Raphaela...,3.72
9,DAVE,Raphaela Edelbauer,25.0,0,None,https://www.booklooker.de/B%C3%BCcher/Raphaela...,3.72


In [30]:
sales_infos = sales_infos.apply(lambda l: pd.DataFrame(l) if type(l)==list else l)

In [31]:
sales_infos.head()
# nd = np.array(sales_infos)

Book Id
28186015    Empty DataFrame
Columns: []
Index: []
44512537    Empty DataFrame
Columns: []
Index: []
4070074     Empty DataFrame
Columns: []
Index: []
56859211    Empty DataFrame
Columns: []
Index: []
16131209    Empty DataFrame
Columns: []
Index: []
dtype: object

In [32]:
sales_infos.iloc[5].iloc[0]

Title                                     Durchbruch bei Stalingrad
Author                                             Heinrich Gerlach
Price                                                          11.0
VSK                                                             6.0
Condition                                 deutliche Gebrauchsspuren
URL               https://www.booklooker.de/B%C3%BCcher/Heinrich...
Average Rating                                                 4.27
Name: 0, dtype: object

Erstmal speichern

In [33]:
pkl_fn = f'sales_infos_2021_recent{TOTAL_NUMBER}_{DATE}.pkl'


In [34]:
sales_infos.to_pickle(pkl_fn)

In [35]:
sales_infos = pd.read_pickle(pkl_fn)


In [36]:
sales_infos.iloc[6]

,Title,Author,Price,VSK,Condition,URL,Average Rating
0,"Dreck - Wie ich meine Familie einpackte, Koch ...",Bill Buford,16.00,2.7,leichte Gebrauchsspuren,https://www.booklooker.de/B%C3%BCcher/Bill-Buf...,3.38
1,"Dreck Wie ich meine Familie einpackte, Koch in...","Buford, Bill (Verfasser)Hübner, Sabine (Überse...",18.39,2.5,wie neu,https://www.booklooker.de/B%C3%BCcher/Bill-Ver...,3.38
2,"Dreck Wie ich meine Familie einpackte, Koch in...","Buford, Bill (Verfasser)Hübner, Sabine (Überse...",18.39,2.5,leichte Gebrauchsspuren,https://www.booklooker.de/B%C3%BCcher/Bill-Ver...,3.38
3,"Dreck Wie ich meine Familie einpackte, Koch in...","Buford, Bill (Verfasser)Hübner, Sabine (Überse...",18.39,2.5,leichte Gebrauchsspuren,https://www.booklooker.de/B%C3%BCcher/Bill-Ver...,3.38
4,"Dreck - Wie ich meine Familie einpackte, Koch ...",Bill Buford,19.40,2.7,wie neu,https://www.booklooker.de/B%C3%BCcher/Bill-Buf...,3.38
5,"Dreck | Wie ich meine Familie einpackte, Koch ...",Bill Buford,26.00,0.0,None,https://www.booklooker.de/B%C3%BCcher/Bill-Buf...,3.38
6,"Dreck | Wie ich meine Familie einpackte, Koch ...",Bill Buford,26.00,0.0,None,https://www.booklooker.de/B%C3%BCcher/Bill-Buf...,3.38
7,Dreck,Bill Buford,26.00,0.0,None,https://www.booklooker.de/B%C3%BCcher/Bill-Buf...,3.38
8,Dreck,Bill Buford,26.00,0.0,None,https://www.booklooker.de/B%C3%BCcher/Bill-Buf...,3.38
9,Dreck,Bill Buford,26.00,0.0,None,https://www.booklooker.de/B%C3%BCcher/Bill-Buf...,3.38


In [37]:
dic = sales_infos.to_dict()
sales_df = pd.concat(dic)

In [38]:
sales_df.iloc[6].iloc[0]

'Durchbruch bei Stalingrad - Die 1949 vom russischen Geheimdienst konfiszierte und nun in russischen Archiven wiederaufgespürte Urfassung des großen Antikriegsromans. Gefunden, herausgegeben und mit einem dokumentarischen Anhang versehen von Carsten Gansel'

In [39]:
book_ids = my_df.index

In [40]:
sales_infos = sales_infos.apply(lambda l: pd.DataFrame(l) if type(l)==list else l)


In [41]:
sales_infos.iloc[0]

""


In [42]:
sales_df.head()

Title  \
29078059 0                          Durchbruch bei Stalingrad   
         1  Durchbruch bei Stalingrad - Die 1949 vom russi...   
         2  Durchbruch bei Stalingrad - Die 1949 vom russi...   
         3  Durchbruch bei Stalingrad 1944. Herausgegeben,...   
         4  Durchbruch bei Stalingrad - Die 1949 vom russi...   

                                                  Author  Price  VSK  \
29078059 0                              Heinrich Gerlach   11.0  6.0   
         1                              Heinrich Gerlach   15.0  2.4   
         2                              Heinrich Gerlach   19.0  4.5   
         3  Gerlach, Heinrich und Carsten Gansel [Hrsg.]   20.0  5.5   
         4                              Heinrich Gerlach   24.0  2.4   

                            Condition  \
29078059 0  deutliche Gebrauchsspuren   
         1    leichte Gebrauchsspuren   
         2    leichte Gebrauchsspuren   
         3             gebraucht; gut   
         4    leichte Gebrauchsspuren   

                                                          URL  Average Rating  
29078059 0  https://www.booklooker.de/B%C3%BCcher/Heinrich...            4.27  
         1  https://www.booklooker.de/B%C3%BCcher/Heinrich...            4.27  
         2  https://www.booklooker.de/B%C3%BCcher/Heinrich...            4.27  
         3  https://www.booklooker.de/B%C3%BCcher/Heinrich...            4.27  
         4  https://www.booklooker.de/B%C3%BCcher/Heinrich...            4.27

In [43]:
sales_df.dtypes

Title              object
Author             object
Price             float64
VSK               float64
Condition          object
URL                object
Average Rating    float64
dtype: object

In [44]:
# sales_df['Price'] = sales_df['Price'].apply(lambda x: x.replace(',','.'))
# sales_df['Price'] = sales_df['Price'].apply(lambda x: non_decimal.sub('', x))
# sales_df['VSK'] = sales_df['VSK'].apply(lambda x: x.replace(',','.'))
# sales_df['VSK'] = sales_df['VSK'].apply(lambda x: non_decimal.sub('', x))
# sales_df.loc[10614].loc[6]['URL']
# sales_df['Price'] = pd.to_numeric(sales_df['Price'])
# sales_df['VSK'] = pd.to_numeric(sales_df['VSK'])
sales_df['Total Price'] = sales_df.fillna(0)['Price'] + sales_df.fillna(0)['VSK']

In [88]:
sales_df['Condition'].replace('None', '', inplace=True)

In [89]:
sales_df.head()

Title                         \
29078059 0                          Durchbruch bei Stalingrad   
         1  Durchbruch bei Stalingrad - Die 1949 vom russi...   
         2  Durchbruch bei Stalingrad - Die 1949 vom russi...   
         3  Durchbruch bei Stalingrad 1944. Herausgegeben,...   
         4  Durchbruch bei Stalingrad - Die 1949 vom russi...   

                               Author                     Price  VSK  \
29078059 0                              Heinrich Gerlach  11.0   6.0   
         1                              Heinrich Gerlach  15.0   2.4   
         2                              Heinrich Gerlach  19.0   4.5   
         3  Gerlach, Heinrich und Carsten Gansel [Hrsg.]  20.0   5.5   
         4                              Heinrich Gerlach  24.0   2.4   

                   Condition           \
29078059 0  deutliche Gebrauchsspuren   
         1    leichte Gebrauchsspuren   
         2    leichte Gebrauchsspuren   
         3             gebraucht; gut   
         4    leichte Gebrauchsspuren   

                                  URL                          Average Rating  \
29078059 0  https://www.booklooker.de/B%C3%BCcher/Heinrich...       4.27        
         1  https://www.booklooker.de/B%C3%BCcher/Heinrich...       4.27        
         2  https://www.booklooker.de/B%C3%BCcher/Heinrich...       4.27        
         3  https://www.booklooker.de/B%C3%BCcher/Heinrich...       4.27        
         4  https://www.booklooker.de/B%C3%BCcher/Heinrich...       4.27        

            Total Price  
29078059 0     17.0      
         1     17.4      
         2     23.5      
         3     25.5      
         4     26.4

In [90]:
df_pkl_fn = f'sales_df_2021_recent{TOTAL_NUMBER}_{DATE}.pkl'

In [91]:
sales_df.to_pickle(df_pkl_fn)

In [92]:
#TODO ggf nochmal erweitern / joinen mit der goodreads Tabelle für Indexierung mittels Titel, etc. (vorallem bei fehelnden Ergebnissen)

In [93]:
sales_df = pd.read_pickle(df_pkl_fn)

In [94]:
sorted_df = sales_df.sort_values(by='Total Price')

In [95]:
by_avg_rating = sales_df.set_index(['Average Rating', sales_df.index, 'Total Price']).sort_index(ascending=[False,True,True])
by_total_price = sales_df.set_index(['Total Price','Average Rating']).sort_index(ascending=[True, False])
by_id = sales_df.set_index([sales_df.index, 'Total Price']).sort_index(ascending=[False, True])


TODO:

1. Sortieren innerhalb BookID
2. nur Bestpreis pro Buch (oder Top 3? oder 1 pro Zustand?)
3. In Flask o.ä. integrieren mit Template für geile Bootstrap-Tabelle mit Filter usw

In [96]:
def export_html(df, fn):
    pd.set_option('colheader_justify', 'center')   # FOR TABLE <th>

    html_string = '''
    <html>
    <head>
        <!-- Required meta tags -->
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1">

    <!-- Bootstrap CSS -->
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.0-beta2/dist/css/bootstrap.min.css" rel="stylesheet" integrity="sha384-BmbxuPwQa2lc/FVzBcNJ7UAyJxM6wuqIj61tLrc4wSX0szH/Ev+nYRRuWlolflfl" crossorigin="anonymous">

    <title>HTML Pandas Dataframe with CSS</title></head>
    <link rel="stylesheet" type="text/css" href="df_style.css"/>
    <body>
        {table}
        <script src="https://cdn.jsdelivr.net/npm/bootstrap@5.0.0-beta2/dist/js/bootstrap.bundle.min.js" integrity="sha384-b5kHyXgcpbZJO/tY9Ul7kGkf1S0CWuKcCD38l8YkeH8z8QjE0GmW1gYU5S9FOnJ0" crossorigin="anonymous"></script>

    </body>
    </html>.
    '''

    # OUTPUT AN HTML FILE
    with open(fn, 'w', encoding='utf-8') as f:
        f.write(html_string.format(table=df.to_html(render_links=True, classes=['table', 'table-hover'])))

In [97]:
export_html(by_avg_rating, f"report_by_rating_{DATE}.html")
export_html(by_total_price, f"report_by_price_{DATE}.html")
export_html(by_id, f"report_by_id_{DATE}.html")

## Visualisierung
- HTML Dashboard (Integration in Flask, FastAPI, Django ?)
- oder super simpel mit Plotly und irgend ner Dashboard JS API (gibts sicher)
- Entweder einfache Tabelle, sortierbar usw.
- oder mit Plots: Barplots: Ein Balken für Neupreis, einen für Gebraucht (mit Whiskers)


In [98]:
def highlight_min(df, color='yellow'):
    '''
    highlight the minimum combined price.
    '''
    attr = 'background-color: {}'.format(color)
    is_min = df == df['Total Price'].min().min()
    return pd.DataFrame(np.where(is_min, attr, ''),
                            index=df.index, columns=df.columns)


In [99]:
# sales_df.style.apply(highlight_min, axis=None)

In [100]:
# sales_df